# Selenium DOC dashboard scraper

THis notebook documents and executes the collection of daily COVID-19 data from the Pennsylvania DOC COVID-19 dashboard. Data is scraped from the DOC dashboard using Selenium and is then processed to conform to a standard dataframe format. 

Data fields collected in this notebook are only for incarcerated people (page 5 of the DOC dashboard), including positive tests, negative tests, pending tests, recovered cases and incarcerated person deaths. 


### Data collection procedure:
1. route selenium driver to DOC dashboard url
2. navigate driver to page 5 of dashboard
3. select date and SCI from dropdown menus
4. right click pie-chart to make table
5. isolate and scrape relevant data fields
6. format and store relevant data fields

In [123]:
# import libraries

import pandas as pd
from bs4 import BeautifulSoup as bs
from urllib import request
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys

import time
import re

### 1. route selenium driver to DOC dashboard URL

In [362]:
dash_url = "https://app.powerbigov.us/view?r=eyJrIjoiNWQ5YTQ4ZWUtY2NjMi00ZWRhLTgyNWQtYzAzNzc5NmYwMGIyIiwidCI6IjQxOGUyODQxLTAxMjgtNGRkNS05YjZjLTQ3ZmM1YTlhMWJkZSJ9"
driver = webdriver.Safari()
driver.get(dash_url)

### 2. navigate driver to page 5 of dashboard

In [363]:
for i in range(4):
    driver.execute_script("document.querySelector('[title=\"Next Page\"]').click()")


### 3. select date and SCI from dropdown menus

In [383]:
# open dropdowns

driver.execute_script('document.querySelector("#pvExplorationHost > div > div > exploration > div > explore-canvas-modern > div > div.canvasFlexBox > div > div.displayArea.disableAnimations.fitToPage > div.visualContainerHost > visual-container-repeat > visual-container-modern:nth-child(7) > transform > div > div:nth-child(3) > div > visual-modern > div > div > div.slicer-content-wrapper > div > i").click()')

driver.execute_script('document.querySelector("#pvExplorationHost > div > div > exploration > div > explore-canvas-modern > div > div.canvasFlexBox > div > div.displayArea.disableAnimations.fitToPage > div.visualContainerHost > visual-container-repeat > visual-container-modern:nth-child(5) > transform > div > div:nth-child(3) > div > visual-modern > div > div > div.slicer-content-wrapper > div").click()')

In [365]:
def roll_through_SCI():
    sci_dropdown_scroll_area = 'document.querySelector("body > div:nth-child(46) > div.slicer-dropdown-content > div > div.slicerBody > div > div.scrollbar-inner.scroll-content.scroll-scrolly_visible")'
    time.sleep(0.1)
    
    height = 200
    sci_list = []
    for d in range(4):
        driver.execute_script(f'{sci_dropdown_scroll_area}.scrollTop = {height*d}')
        
        time.sleep(0.1)
        
        sci_scroll_area = driver.execute_script(f'return {sci_dropdown_scroll_area}')
        current_sci = sci_scroll_area.find_elements_by_class_name('slicerText')
        
        for i in current_sci:
            sci = i.get_attribute('title')
            if sci not in sci_list:
                sci_list.append(sci)
                driver.execute_script(f"document.querySelector('[title=\"{sci}\"]').click()")
                # Add in get data and go back
                
        

In [366]:
def roll_through_dates():
    date_dropdown_scroll_area = 'document.querySelector("body > div:nth-child(47) > div.slicer-dropdown-content > div > div.slicerBody > div > div.scrollbar-inner.scroll-content.scroll-scrolly_visible")'
    
    height = 200
    date_list = []
    for d in range(1):
        driver.execute_script(f'{date_dropdown_scroll_area}.scrollTop = {height*d}')
        
        time.sleep(0.1)
        
        date_scroll_area = driver.execute_script(f'return {date_dropdown_scroll_area}')
        current_dates = date_scroll_area.find_elements_by_class_name('slicerText')
    
        for i in current_dates[:3]:
            date = i.get_attribute('title')
            if date not in date_list:
                date_list.append(date)
                driver.execute_script(f"document.querySelector('[title=\"{date}\"]').click()")
                time.sleep(0.1)
                roll_through_SCI()
                

### 4. right click pie-chart to make table

In [262]:
def right_click_donut():
    action = ActionChains(driver)
    action.context_click(driver.find_element_by_class_name('donutChart')).perform()
    driver.execute_script("document.querySelector('[title=\"Show as a table\"]').click()")


### 5. isolate and scrape relevant data fields

## TODOs

* pull data out of current table
* Need to format data
* go back to original format and do again